In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import os
import time

In [2]:
# Thiết lập đường dẫn động cho các file nguồn dữ liệu nhân sự
# Vai trò: Xác định thư mục gốc theo user (giúp chạy được trên nhiều máy khác nhau trong team), 
# và khai báo đường dẫn đến các file dữ liệu quan trọng vào một dictionary. 
# Bước này giúp các thao tác đọc dữ liệu tiếp theo (Extract) được quản lý tập trung, dễ bảo trì khi đổi đường dẫn.
first_glob_1 = "C:/Users/huuchinh.nguyen"
first_glob_2 = "C:/Users/ADMIN"

if os.path.exists(first_glob_1):
    first_glob = first_glob_1
elif os.path.exists(first_glob_2):
    first_glob = first_glob_2
else:
    raise FileNotFoundError(f"Neither {first_glob_1} nor {first_glob_2} exists.")

folder_paths = {
    "hc_staffing":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Headcount/HC Master Database - 2025.xlsx',
    "hc_active":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Headcount/HC Master Database - 2025.xlsx',
    "hc_inactive":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Headcount/HC Master Database - 2025.xlsx',
    "date_format":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Headcount/HC Master Database - 2025.xlsx',
    "hc_extend_by_week":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Headcount/HC Extend by Month/'
}

In [3]:
# Đọc, tiền xử lý, kết hợp dữ liệu nhân sự, và tạo bảng mở rộng phục vụ phân tích động headcount
# Vai trò tổng quan: 
# 1. Đọc dữ liệu từ các sheet quan trọng (Staffing_Records, Active, Inactive, Date_Format) - bước Extract.
# 2. Lựa chọn các trường thông tin cần thiết cho dashboard headcount từ bảng Active và Inactive.
# 3. Combine (kết hợp) dữ liệu active và inactive để chuẩn hóa toàn bộ workforce.
# 4. Tạo bảng headcount mở rộng (expand theo từng ngày và từng OracleID) giúp theo dõi trạng thái nhân sự theo thời gian thực.
# 5. Chuẩn hóa các trường ngày tháng, xử lý dữ liệu thiếu, loại bỏ dòng trùng lặp hoặc không hợp lệ.
# 6. Output là một bảng động ('HC_Data_Edition') đã sạch, phục vụ phân tích/lập báo cáo linh hoạt về nhân sự call center.


# 1. Đọc dữ liệu từ các sheet chính (Extract)
staffing_records = pd.read_excel(folder_paths['hc_staffing'], sheet_name="Staffing_Records")
hc_active = pd.read_excel(folder_paths['hc_active'], sheet_name="Active")
hc_inactive = pd.read_excel(folder_paths['hc_inactive'], sheet_name="Inactive")
date_format = pd.read_excel(folder_paths['date_format'], sheet_name="Date_Format")

# 2. Lựa chọn các cột cần thiết cho dashboard từ Active
hc_active = hc_active[['Site', 'Queue Group', 'CSG Joining Date', 'Location', 'People ID',
       'IEX ID', 'OracleID', 'Employee Name', 'Alias', 'Gender',
       'Contract Start Date', 'Contract End Date', 'Designation','Grade', 'LOB',
       'Role', 'Multiple Chat Effective Date', 'Primary role',
       'Secondary role', 'TL ID', 'Supervisor Name', 'Manager/OM Name',
       'Email Id', 'Wave', 'CCT Training', 'Lodging - Training start date',
       'Lodging - Training end date', 'Lodging - Nesting Date',
       'Lodging - Certification Date (Original)',
       'Lodging - Certification Date (Actual)',
       'Lodging - Certification status', 'Non-Lodging - Training start date',
       'Non-Lodging - Training end date', 'Non-Lodging - Nesting Date',
       'Non-Lodging - Certification Date',
       'Non-Lodging - Certification Date (Actual)',
       'Non-Lodging - Certification status', 'Production Date', 'AON',
       'Tenure', 'Nationality', 'MSA']]

# 3. Lựa chọn các cột cần thiết cho dashboard từ Inactive (khác đôi chút về tên Manager/OM Name vs Manager Name
hc_inactive = hc_inactive[['Site', 'Queue Group', 'CSG Joining Data', 'Location', 'People ID',
       'IEX ID', 'OracleID', 'Employee Name', 'Alias', 'Gender',
       'Contract Start Date', 'Contract End Date', 'Designation','Grade', 'LOB',
       'Role', 'Multiple Chat Effective Date', 'Primary role',
       'Secondary role', 'TL ID', 'Supervisor Name', 'Manager Name',
       'Email Id', 'Wave', 'CCT Training', 'Lodging - Training start date ',
       'Lodging - Training end date', 'Lodging - Nesting Date',
       'Lodging - Certification Date (Original)',
       'Lodging - Certification Date (Actual)',
       'Lodging - Certification status', 'Non-Lodging - Training start date ',
       'Non-Lodging - Training end date', 'Non-Lodging - Nesting Date',
       'Non-Lodging - Certification Date',
       'Non-Lodging - Certification Date (Actual)',
       'Non-Lodging - Certification status', 'Production Date', 'AON',
       'Tenure', 'Nationality', 'MSA', 'LWD/Movement', 'Attrition Type',
       'Reason for Attrition', 'Month', 'Week Ending']]

# 4. Combine hai bảng active và inactive
hc_combined = pd.concat([hc_active, hc_inactive], ignore_index=True)

# 5. Tạo list ngày cần phân tích (từ 2023-12-01 đến 15 ngày sau hiện tại)
start_date = datetime(2023, 12, 1)
end_date = datetime.now() + timedelta(days=15)
date_list = pd.date_range(start_date, end_date)

# 6. Tạo bảng mở rộng: mỗi ngày, mỗi OracleID có một dòng (phục vụ theo dõi biến động headcount theo từng ngày)
date_table = pd.DataFrame(date_list, columns=['Date'])
expanded_hc = pd.DataFrame(
    [(date, oracle_id) for oracle_id in hc_combined['OracleID'] for date in date_list],
    columns=['Date', 'OracleID']
)

# 7. Merge bảng expanded với dữ liệu gốc để gắn thông tin nhân viên lên từng ngày
HC_Data_Edition = expanded_hc.merge(hc_combined, on='OracleID', how='left')

# 8. Danh sách các cột ngày tháng cần chuyển đổi định dạng datetime
date_columns = ['Contract Start Date', 'Contract End Date', 'Multiple Chat Effective Date', 
                'CCT Training', 'Lodging - Training start date', 'Lodging - Training end date', 
                'Lodging - Nesting Date', 'Lodging - Certification Date (Original)', 
                'Lodging - Certification Date (Actual)', 'Non-Lodging - Training start date', 
                'Non-Lodging - Training end date', 'Non-Lodging - Nesting Date', 
                'Non-Lodging - Certification Date', 'Non-Lodging - Certification Date (Actual)', 
                'Production Date', 'LWD/Movement', 'CSG Joining Date']

# 9. Chuẩn hóa định dạng ngày tháng cho các trường ngày trong bảng
for column in date_columns:HC_Data_Edition[column] = pd.to_datetime(HC_Data_Edition[column], errors='coerce')

# 10. Xử lý dữ liệu thiếu (missing) và loại bỏ dòng trùng lặp, không hợp lệ
HC_Data_Edition.fillna(pd.NA, inplace=True)
HC_Data_Edition = HC_Data_Edition.sort_values(by=["OracleID", "Date"], ascending=[True, True])
HC_Data_Edition = HC_Data_Edition.drop_duplicates(subset=['Date', 'OracleID'])
HC_Data_Edition = HC_Data_Edition[HC_Data_Edition['OracleID'].notnull()]

# 11. Kiểm tra các cột dữ liệu
print("Kiểm tra các cột dữ liệu:\n", HC_Data_Edition.columns, "\nKiểm tra xong")

# 12. Kiểm tra các trường hợp OracleID bị thiếu thông tin nhân viên (phục vụ debug)
print("\nKiểm tra các trường hợp bị thiếu OracleID")
HC_Data_Edition[HC_Data_Edition['Employee Name'].isna()][['Date','OracleID','IEX ID','Employee Name']].head(50)
print("Kiểm tra xong")

# 13. Output: bảng dữ liệu nhân sự đã mở rộng theo từng ngày và OracleID, dùng cho các bước phân tích tiếp theo
print("\nBảng HC_Data_Edition")
HC_Data_Edition

c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:223: UserWarning: Cell Q576 is marked as a date but the serial value 103145248 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:223: UserWarning: Cell Q716 is marked as a date but the serial value 103145248 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


Kiểm tra các cột dữ liệu:
 Index(['Date', 'OracleID', 'Site', 'Queue Group', 'CSG Joining Date',
       'Location', 'People ID', 'IEX ID', 'Employee Name', 'Alias', 'Gender',
       'Contract Start Date', 'Contract End Date', 'Designation', 'Grade',
       'LOB', 'Role', 'Multiple Chat Effective Date', 'Primary role',
       'Secondary role', 'TL ID', 'Supervisor Name', 'Manager/OM Name',
       'Email Id', 'Wave', 'CCT Training', 'Lodging - Training start date',
       'Lodging - Training end date', 'Lodging - Nesting Date',
       'Lodging - Certification Date (Original)',
       'Lodging - Certification Date (Actual)',
       'Lodging - Certification status', 'Non-Lodging - Training start date',
       'Non-Lodging - Training end date', 'Non-Lodging - Nesting Date',
       'Non-Lodging - Certification Date',
       'Non-Lodging - Certification Date (Actual)',
       'Non-Lodging - Certification status', 'Production Date', 'AON',
       'Tenure', 'Nationality', 'MSA', 'CSG Joining Da

,Date,OracleID,Site,Queue Group,CSG Joining Date,Location,People ID,IEX ID,Employee Name,Alias,...,MSA,CSG Joining Data,Manager Name,Lodging - Training start date,Non-Lodging - Training start date,LWD/Movement,Attrition Type,Reason for Attrition,Month,Week Ending
195603,2023-12-01,1081503.0,-,-,NaT,Vietnam,178879477.0,NaN,TRAN PHAM NGOC VAN,Van,...,Expedia,-,Sunny Munjal,-,-,2024-05-02,<NA>,Movement to Etraveli,May'24,2024-05-05 00:00:00
195604,2023-12-02,1081503.0,-,-,NaT,Vietnam,178879477.0,NaN,TRAN PHAM NGOC VAN,Van,...,Expedia,-,Sunny Munjal,-,-,2024-05-02,<NA>,Movement to Etraveli,May'24,2024-05-05 00:00:00
195605,2023-12-03,1081503.0,-,-,NaT,Vietnam,178879477.0,NaN,TRAN PHAM NGOC VAN,Van,...,Expedia,-,Sunny Munjal,-,-,2024-05-02,<NA>,Movement to Etraveli,May'24,2024-05-05 00:00:00
195606,2023-12-04,1081503.0,-,-,NaT,Vietnam,178879477.0,NaN,TRAN PHAM NGOC VAN,Van,...,Expedia,-,Sunny Munjal,-,-,2024-05-02,<NA>,Movement to Etraveli,May'24,2024-05-05 00:00:00
195607,2023-12-05,1081503.0,-,-,NaT,Vietnam,178879477.0,NaN,TRAN PHAM NGOC VAN,Van,...,Expedia,-,Sunny Munjal,-,-,2024-05-02,<NA>,Movement to Etraveli,May'24,2024-05-05 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429860,2025-06-25,103202199.0,ONEHUB,-,NaT,Vietnam,917303552.0,3100099.0,LE VAN QUANG,-,...,Expedia,-,Patar Kirpan,-,-,2025-05-17,Involuntary,Terminate Employee > Involuntary > Reduction i...,May'25,2025-05-18 00:00:00
429861,2025-06-26,103202199.0,ONEHUB,-,NaT,Vietnam,917303552.0,3100099.0,LE VAN QUANG,-,...,Expedia,-,Patar Kirpan,-,-,2025-05-17,Involuntary,Terminate Employee > Involuntary > Reduction i...,May'25,2025-05-18 00:00:00
429862,2025-06-27,103202199.0,ONEHUB,-,NaT,Vietnam,917303552.0,3100099.0,LE VAN QUANG,-,...,Expedia,-,Patar Kirpan,-,-,2025-05-17,Involuntary,Terminate Employee > Involuntary > Reduction i...,May'25,2025-05-18 00:00:00
429863,2025-06-28,103202199.0,ONEHUB,-,NaT,Vietnam,917303552.0,3100099.0,LE VAN QUANG,-,...,Expedia,-,Patar Kirpan,-,-,2025-05-17,Involuntary,Terminate Employee > Involuntary > Reduction i...,May'25,2025-05-18 00:00:00


In [4]:
# Mapping dữ liệu headcount từng ngày với lịch sử thay đổi hiệu lực nhân sự (Effective Date)
# Vai trò tổng quan:
# 1. Chuẩn hóa cột ngày tháng cho dữ liệu phân tích.
# 2. Kết hợp (mapping) giữa bảng headcount từng ngày và bảng staffing_records để gắn trạng thái nhân sự tại từng thời điểm, 
# dựa trên ngày hiệu lực (Effective Date).
# 3. Xử lý logic "ffill" (forward fill) để truy ngược lịch sử trạng thái nhân sự cho từng OracleID tại từng ngày phân tích.
# 4. Merge dữ liệu nghỉ việc, lý do nghỉ vào bảng staffing, phục vụ phân tích churn, attrition.
# 5. Chuẩn bị bảng dữ liệu sạch, truy vết đầy đủ biến động nhân sự (phục vụ dashboard headcount động và phân tích attrition).

# 1. Chuẩn hóa cột ngày tháng
HC_Data_Edition["Date"] = pd.to_datetime(HC_Data_Edition["Date"], errors='coerce')
staffing_records["Effective Date"] = pd.to_datetime(staffing_records["Effective Date"], errors='coerce')

# 2. Lấy cặp (Date, OracleID) để mapping với Effective Date trong staffing_records
HC_Data_Edition_Only_ID = HC_Data_Edition[['Date','OracleID']]
staffing_records_date = staffing_records[['Effective Date', 'OracleID']]
staffing_records_date = staffing_records_date.rename(columns={'OracleID': 'Changed.OracleID'})

# 3. Merge để lấy các OracleID tồn tại trong staffing_records
merged_id = pd.merge(HC_Data_Edition_Only_ID, staffing_records_date[['Changed.OracleID']], how='left', 
                     left_on=['OracleID'], right_on=['Changed.OracleID'])
merged_id = merged_id[merged_id['Changed.OracleID'].notnull()]
merged_id = merged_id.drop(columns=['Changed.OracleID'])

# 4. Merge theo cả Date và OracleID để tìm đúng mốc Effective Date cho mỗi snapshot (ngày)
merged_effective_date = pd.merge(merged_id, staffing_records_date[['Effective Date','Changed.OracleID']], 
                                 how='left', left_on=['Date','OracleID'], right_on=['Effective Date','Changed.OracleID'])
merged_effective_date = merged_effective_date.sort_values(by=["OracleID", "Date"], ascending=[True, True])
merged_effective_date = merged_effective_date.drop_duplicates(subset=['Date', 'OracleID', 'Effective Date', 'Changed.OracleID'])

# 5. Điền đầy đủ các mốc Effective Date cho các ngày phân tích bằng cách ffill (forward fill)
merged_effective_date['Effective Date'] = merged_effective_date['Effective Date'].ffill()

# 6. Đảm bảo chỉ giữ lại các dòng mà Effective Date không vượt quá ngày phân tích hiện tại (đúng nghiệp vụ HR)
merged_effective_date['Effective Date'] = merged_effective_date.apply(
    lambda row: row['Effective Date'] if row['Effective Date'] <= row['Date'] else pd.NaT,axis=1
    )
merged_effective_date = merged_effective_date[merged_effective_date['Effective Date'].notnull()]
merged_effective_date = merged_effective_date.drop(columns=['Changed.OracleID'])

# 7. Kiểm tra lại các cột sau mapping
print("Kiểm tra các cột\n",merged_effective_date.columns,"\nKiểm tra xong\n")

# 8. Merge bảng đã mapping Effective Date với staffing_records để lấy đầy đủ thông tin nhân sự tại thời điểm đó
merged_staffing_records = pd.merge(merged_effective_date, staffing_records, how='left', on=['Effective Date','OracleID'])

# 9. Merge thêm thông tin nghỉ việc, lý do nghỉ từ HC_Data_Edition (phục vụ phân tích attrition)
HC_Data_Edition_Inactive = HC_Data_Edition.filter(items=['Date','Site', 'Queue Group', 'CSG Joining Date', 
                                                         'OracleID', 'LWD/Movement', 'Attrition Type','Reason for Attrition'])
merged_inactive_cols = pd.merge(merged_staffing_records, HC_Data_Edition_Inactive, on=['Date','OracleID',], how='left')
staffing_records = merged_inactive_cols

# 10. Merge lại với HC_Data_Edition để xác định các ngày mà OracleID không thay đổi trạng thái (không có Effective Date mới)
merged_staffing_records = HC_Data_Edition.merge(
    staffing_records[['Date', 'OracleID', 'Effective Date']],
    how='left',
    on=['Date', 'OracleID']
)

HC_Data_Edition_not_change = merged_staffing_records[merged_staffing_records['Effective Date'].isna()]

# 11. Kết hợp dữ liệu đã có trạng thái mới (hiệu lực) và dữ liệu không đổi trạng thái
hc_extend = pd.concat([HC_Data_Edition_not_change, staffing_records], ignore_index=True)
hc_extend = hc_extend.drop(columns=['Effective Date'])

Kiểm tra các cột
 Index(['Date', 'OracleID', 'Effective Date'], dtype='object') 
Kiểm tra xong



In [5]:
# Chuẩn hóa trường, tính toán trạng thái active và tenure cho từng nhân sự mỗi ngày
# Vai trò tổng quan:
# 1. Chuẩn hóa, rút gọn, đổi tên các trường ngày tháng/phân loại, giúp đồng bộ field giữa các nhóm dữ liệu.
# 2. Tính toán các mốc thời gian quan trọng cho mỗi nhân sự (Production, Nesting, Training...) dựa trên ngày thực tế và nghiệp vụ.
# 3. Xây dựng logic tính trạng thái chi tiết (Detail Status) và tính số ngày ở trạng thái Production (AON - Age On Nesting/Production).
# 4. Gán nhãn tenure động (0-30/30-60/60-90/>90 Days) cho từng nhân sự và từng ngày, tách riêng Lodging & Non-Lodging.
# 5. Chuẩn bị bảng cuối cùng cho dashboard nhân sự: hiển thị chính xác trạng thái, thời gian công tác, lý do nghỉ việc, phù hợp chuẩn phân tích động cho call center.

# 1. Loại bỏ các cột không cần thiết để chuẩn hóa bảng
hc_extend = hc_extend.drop(columns=["Lodging - Training end date", "Non-Lodging - Training end date"])

# 2. Đổi tên các cột để dễ sử dụng về sau, đồng bộ field giữa các nhóm dữ liệu (giảm rối mắt, tăng readability)
hc_extend = hc_extend.rename(columns={
    "Lodging - Training start date": "Lodging Training",
    "Lodging - Nesting Date": "Lodging Nesting",
    "Lodging - Certification Date (Original)": "Lodging Original",
    "Lodging - Certification Date (Actual)": "Lodging Actual",
    "Lodging - Certification status": "Lodging",
    "Non-Lodging - Training start date": "NonLodging Training",
    "Non-Lodging - Nesting Date": "NonLodging Nesting",
    "Non-Lodging - Certification Date": "NonLodging Original",
    "Non-Lodging - Certification Date (Actual)": "NonLodging Actual",
    "Non-Lodging - Certification status": "NonLodging",
    "Production Date": "Production"
})

# 3. Chuẩn hóa lại định dạng ngày tháng cho tất cả các cột ngày quan trọng
date_columns = [
    "Date", "CCT Training", "Lodging Training", "Lodging Nesting", "Lodging Original", "Lodging Actual",
    "NonLodging Training", "NonLodging Nesting", "NonLodging Actual", "Production", "NonLodging Original",
    "Multiple Chat Effective Date", "LWD/Movement", "CSG Joining Date"
]
for col in date_columns:
    hc_extend[col] = pd.to_datetime(hc_extend[col], errors='coerce')

# 4. Tạo cột mới: ngày trong tuần của ngày chứng chỉ (Certification) thực tế, phục vụ tính logic nghiệp vụ (điểm chuẩn Production)
hc_extend['Lodging Days'] = hc_extend['Lodging Actual'].dt.day_name()
hc_extend['NonLodging Days'] = hc_extend['NonLodging Actual'].dt.day_name()

# 5. Hàm phụ: lấy ngày Monday liền sau một ngày bất kỳ (áp dụng cho logic xác định production)
def get_next_monday(day_name, date):
    if pd.isna(date):
        return pd.NaT
    return {
        "Monday": date,
        "Tuesday": date + pd.Timedelta(days=6),
        "Wednesday": date + pd.Timedelta(days=5),
        "Thursday": date + pd.Timedelta(days=4),
        "Friday": date + pd.Timedelta(days=3),
        "Saturday": date + pd.Timedelta(days=2),
        "Sunday": date + pd.Timedelta(days=1)
    }.get(day_name, pd.NaT)

# 6. Xác định ngày Production thật sự dựa vào ngày chứng chỉ (Actual) và logic Monday chuẩn theo quy định nghiệp vụ
hc_extend["Next Monday Lodging"] = hc_extend.apply(lambda row: get_next_monday(row["Lodging Days"], row["Lodging Actual"]), axis=1)
hc_extend["Next Monday NonLodging"] = hc_extend.apply(lambda row: get_next_monday(row["NonLodging Days"], row["NonLodging Actual"]), axis=1)

# 7. Tính ngày kết thúc Nesting mở rộng (thêm 14 ngày so với ngày Nesting, trừ trường hợp trùng ngày Production)
hc_extend["Lodging Extended Nesting"] = np.where(
    hc_extend["Lodging Nesting"] + pd.Timedelta(days=14) == hc_extend["Next Monday Lodging"],
    pd.NaT,
    hc_extend["Lodging Nesting"] + pd.Timedelta(days=14)
)

# 8. Xác định ngày bắt đầu Production thực tế cho từng LOB
hc_extend["NonLodging Extended Nesting"] = np.where(
    hc_extend["NonLodging Nesting"] + pd.Timedelta(days=14) == hc_extend["Next Monday NonLodging"],
    pd.NaT,
    hc_extend["NonLodging Nesting"] + pd.Timedelta(days=14)
)

hc_extend["Lodging Production"] = hc_extend["Next Monday Lodging"]
hc_extend["NonLodging Production"] = hc_extend["Next Monday NonLodging"]

# 9. Loại bỏ các cột phụ trợ không cần thiết sau tính toán (giúp bảng gọn, tránh rối cho dashboard)
cols_to_remove = [
    "Next Monday Lodging", "Next Monday NonLodging", "Lodging Days", "NonLodging Days",
    "Lodging Original", "NonLodging Original", "Lodging Actual", "Lodging",
    "NonLodging Actual", "NonLodging", "Contract Start Date", "Contract End Date"
]
hc_extend = hc_extend.drop(columns=[col for col in cols_to_remove if col in hc_extend.columns])

# 10. Đặt lại thứ tự các cột để bảng dễ nhìn, đúng chuẩn báo cáo/dashboard
column_order = [
    "Date",'Site', 'Queue Group', "OracleID", "People ID", "IEX ID", "Employee Name", "Alias", "Gender", "Designation","Grade", "LOB",
    "Multiple Chat Effective Date","Role", "Primary role", "Secondary role", "TL ID", "Supervisor Name",
    "Manager/OM Name", "Email Id", "Wave", "CCT Training", "Lodging Training", "Lodging Nesting",
    "Lodging Extended Nesting", "Lodging Production", "NonLodging Training", "NonLodging Nesting",
    "NonLodging Extended Nesting", "NonLodging Production", "Production", "Attrition Type", 'Reason for Attrition', 'ATT_Reason_LV3', "LWD/Movement", "CSG Joining Date"
]
hc_extend = hc_extend[[col for col in column_order if col in hc_extend.columns]]

# 11. Sắp xếp bảng theo ngày
hc_extend = hc_extend.sort_values("Date")

# 12. Đảm bảo lại các cột ngày ở định dạng datetime
date_cols = [
    "Date", "NonLodging Production", "NonLodging Extended Nesting", "NonLodging Nesting", "NonLodging Training",
    "Lodging Production", "Lodging Extended Nesting", "Lodging Nesting", "Lodging Training", "CCT Training"
]

for col in date_cols:
    hc_extend[col] = pd.to_datetime(hc_extend[col], errors='coerce')

# 13. Hàm xác định trạng thái động chi tiết của nhân viên theo từng ngày    
def get_detail_status(row):
    d = row["Date"]
    nl_p = row["NonLodging Production"]
    nl_ex = row["NonLodging Extended Nesting"]
    nl_n = row["NonLodging Nesting"]
    nl_t = row["NonLodging Training"]
    l_p = row["Lodging Production"]
    l_ex = row["Lodging Extended Nesting"]
    l_n = row["Lodging Nesting"]
    l_t = row["Lodging Training"]
    cct = row["CCT Training"]

    if pd.notna(nl_p) and pd.notna(d) and d >= nl_p: return "Non Lodging Production"
    if pd.notna(nl_ex) and pd.notna(d) and d >= nl_ex and pd.isna(nl_p): return "Non Lodging Extended Nesting"
    if pd.notna(nl_ex) and pd.notna(d) and nl_ex <= d < nl_p: return "Non Lodging Extended Nesting"
    if pd.notna(nl_n) and pd.notna(d) and d >= nl_n and pd.isna(nl_ex): return "Non Lodging Nesting"
    if pd.notna(nl_n) and pd.notna(d) and nl_n <= d < nl_ex: return "Non Lodging Nesting"
    if pd.notna(nl_t) and pd.notna(d) and d >= nl_t and pd.isna(nl_n): return "Non Lodging Training"
    if pd.notna(nl_t) and pd.notna(d) and nl_t <= d < nl_n: return "Non Lodging Training"
    if pd.notna(l_p) and pd.notna(d) and d >= l_p and pd.isna(nl_t): return "Lodging Production"
    if pd.notna(l_p) and pd.notna(d) and l_p <= d < nl_t: return "Lodging Production"
    if pd.notna(l_ex) and pd.notna(d) and d >= l_ex and pd.isna(l_p): return "Lodging Extended Nesting"
    if pd.notna(l_ex) and pd.notna(d) and l_ex <= d < l_p: return "Lodging Extended Nesting"
    if pd.notna(l_n) and pd.notna(d) and d >= l_n and pd.isna(l_ex): return "Lodging Nesting"
    if pd.notna(l_n) and pd.notna(d) and l_n <= d < l_ex: return "Lodging Nesting"
    if pd.notna(l_t) and pd.notna(d) and d >= l_t and pd.isna(l_n): return "Lodging Training"
    if pd.notna(l_t) and pd.notna(d) and l_t <= d < l_n: return "Lodging Training"
    if pd.notna(cct) and pd.notna(d) and d >= cct and pd.isna(l_t): return "CCT Training"
    if pd.notna(cct) and pd.notna(d) and cct <= d < l_t: return "CCT Training"
    if pd.notna(cct) and pd.notna(d) and d < cct: return "Unavailable"

    return "-"

hc_extend["Detail Status"] = hc_extend.apply(get_detail_status, axis=1)

# 14. Hàm tính số ngày đã ở trạng thái Production ("Age on Nesting/Production" - AON)
def get_aon(row):
    if row["Detail Status"] == "Lodging Production":
        if pd.isna(row["Lodging Production"]) or row["Date"] < row["Lodging Production"]:
            return None
        else:
            return (row["Date"] - row["Lodging Production"]).days
    elif row["Detail Status"] == "Non Lodging Production":
        if pd.isna(row["NonLodging Production"]) or row["Date"] < row["NonLodging Production"]:
            return None
        else:
            return (row["Date"] - row["NonLodging Production"]).days
    else:
        return None

hc_extend["AON"] = hc_extend.apply(get_aon, axis=1)

# 15. Hàm gán nhãn tenure động cho Lodging (0-30, 30-60, 60-90, >90 ngày)
def get_lg_tenure(row):
    if row["Detail Status"] in ["Lodging Nesting", "Lodging Extended Nesting"]:
        return "Nesting"
    elif pd.notna(row["Lodging Production"]) and row["Date"] >= row["Lodging Production"]:
        aon = row["AON"]
        if pd.isna(aon) or aon == "-":
            return "-"
        elif 0 <= aon <= 30:
            return "0-30 Days"
        elif 30 < aon <= 60:
            return "30-60 Days"
        elif 60 < aon <= 90:
            return "60-90 Days"
        elif aon > 90:
            return "> 90 Days"
    return None

hc_extend["LG Tenure"] = hc_extend.apply(get_lg_tenure, axis=1)

# 16. Hàm gán nhãn tenure động cho Non-Lodging (0-30, 30-60, 60-90, >90 ngày)
def get_nl_tenure(row):
    if row["Detail Status"] in ["Non Lodging Nesting", "Non Lodging Extended Nesting"]:
        return "Nesting"
    elif pd.notna(row["NonLodging Production"]) and row["Date"] >= row["NonLodging Production"]:
        aon = row["AON"]
        if pd.isna(aon) or aon == "-":
            return "-"
        elif 0 <= aon <= 30:
            return "0-30 Days"
        elif 30 < aon <= 60:
            return "30-60 Days"
        elif 60 < aon <= 90:
            return "60-90 Days"
        elif aon > 90:
            return "> 90 Days"
    return None

hc_extend["NL Tenure"] = hc_extend.apply(get_nl_tenure, axis=1)

# 17. Kiểm tra lại các cột sau chuẩn hóa
print(hc_extend.columns)

Index(['Date', 'Site', 'Queue Group', 'OracleID', 'People ID', 'IEX ID',
       'Employee Name', 'Alias', 'Gender', 'Designation', 'Grade', 'LOB',
       'Multiple Chat Effective Date', 'Role', 'Primary role',
       'Secondary role', 'TL ID', 'Supervisor Name', 'Manager/OM Name',
       'Email Id', 'Wave', 'CCT Training', 'Lodging Training',
       'Lodging Nesting', 'Lodging Extended Nesting', 'Lodging Production',
       'NonLodging Training', 'NonLodging Nesting',
       'NonLodging Extended Nesting', 'NonLodging Production', 'Production',
       'Attrition Type', 'Reason for Attrition', 'LWD/Movement',
       'CSG Joining Date', 'Detail Status', 'AON', 'LG Tenure', 'NL Tenure'],
      dtype='object')


In [6]:
# 1. Kiểm tra trạng thái Lodging/NonLodging cho nhân sự cụ thể (debug cá nhân, có thể bỏ khi chạy production)
hc_extend[hc_extend['Employee Name']=="NGUYEN THI NGOC ANH"][['Lodging Training',
       'Lodging Nesting', 'Lodging Extended Nesting', 'Lodging Production',
       'NonLodging Training', 'NonLodging Nesting',
       'NonLodging Extended Nesting', 'NonLodging Production', 'Production']]

,Lodging Training,Lodging Nesting,Lodging Extended Nesting,Lodging Production,NonLodging Training,NonLodging Nesting,NonLodging Extended Nesting,NonLodging Production,Production
126020,NaT,2024-06-10,NaT,2024-06-24,NaT,2025-02-26,2025-03-12,2025-03-17,2024-06-24
126021,NaT,2024-06-10,NaT,2024-06-24,NaT,2025-02-26,2025-03-12,2025-03-17,2024-06-24
126022,NaT,2024-06-10,NaT,2024-06-24,NaT,2025-02-26,2025-03-12,2025-03-17,2024-06-24
126023,NaT,2024-06-10,NaT,2024-06-24,NaT,2025-02-26,2025-03-12,2025-03-17,2024-06-24
126024,NaT,2024-06-10,NaT,2024-06-24,NaT,2025-02-26,2025-03-12,2025-03-17,2024-06-24
...,...,...,...,...,...,...,...,...,...
401016,2024-05-20,2024-06-10,NaT,2024-06-24,2025-02-04,2025-02-26,2025-03-12,2025-03-17,2024-06-24
401017,2024-05-20,2024-06-10,NaT,2024-06-24,2025-02-04,2025-02-26,2025-03-12,2025-03-17,2024-06-24
401018,2024-05-20,2024-06-10,NaT,2024-06-24,2025-02-04,2025-02-26,2025-03-12,2025-03-17,2024-06-24
401019,2024-05-20,2024-06-10,NaT,2024-06-24,2025-02-04,2025-02-26,2025-03-12,2025-03-17,2024-06-24


In [7]:
print(hc_extend.columns.tolist())


['Date', 'Site', 'Queue Group', 'OracleID', 'People ID', 'IEX ID', 'Employee Name', 'Alias', 'Gender', 'Designation', 'Grade', 'LOB', 'Multiple Chat Effective Date', 'Role', 'Primary role', 'Secondary role', 'TL ID', 'Supervisor Name', 'Manager/OM Name', 'Email Id', 'Wave', 'CCT Training', 'Lodging Training', 'Lodging Nesting', 'Lodging Extended Nesting', 'Lodging Production', 'NonLodging Training', 'NonLodging Nesting', 'NonLodging Extended Nesting', 'NonLodging Production', 'Production', 'Attrition Type', 'Reason for Attrition', 'LWD/Movement', 'CSG Joining Date', 'Detail Status', 'AON', 'LG Tenure', 'NL Tenure']


In [8]:
# Chuẩn hóa trạng thái động, enrich thuộc tính phân tích và chuẩn bị bảng final headcount động cho dashboard
# Vai trò tổng quan:
# 1. Tính trạng thái "Terminated" và "Active" cho từng nhân viên tại từng ngày dựa trên ngày nghỉ (LWD/Movement) 
# và ngày bắt đầu training.
# 2. Tính concurrency (số lượng task đồng thời) dựa trên detail status và logic Multiple Chat.
# 3. Gán nhóm LOB (Line of Business) 2 và 3 động theo trạng thái, phân biệt Chat/Voice, Lodging/Non Lodging, Nesting/Production.
# 4. Tối ưu lại các trường đặc biệt phục vụ dashboard: performance calculation, loại bỏ những dòng không phân tích, 
# xóa dòng trùng, enrich với bảng ngày (date_format).
# 5. Chuẩn hóa Stage (In Training/Nesting/Production), nhóm lại lý do nghỉ việc cấp 3, tính HC Open/Close từng tuần/tháng.
# 6. Tính toán các chỉ số đặc biệt như ATT (attrition), Movement, concurrency, CSG.
# 7. Đặt lại thứ tự cột final, đảm bảo bảng xuất ra sẵn sàng cho Power BI/Tableau hoặc dashboard Python.

# 2. Tính trạng thái động: "Terminated" nếu qua ngày nghỉ, "Active" nếu đã bắt đầu training, còn lại None
hc_extend["Status"] = np.where(
    (hc_extend["Date"] >= hc_extend["LWD/Movement"]) & (hc_extend["LWD/Movement"].notna()),
    "Terminated",
    np.where(hc_extend["Date"] >= hc_extend["CCT Training"], "Active", None)
).astype(object)

# 3. Tính concurrency: Nếu detail status là Production/Nesting thì concurrency là 2 nếu có hiệu lực Multiple Chat, 
# ngược lại là 1; còn lại là 0
hc_extend["Concurrency"] = np.where(
    hc_extend["Detail Status"].str.contains("Production|Nesting", na=False),
    np.where(
        (hc_extend["Multiple Chat Effective Date"].notna()) 
        & (hc_extend["Date"] >= hc_extend["Multiple Chat Effective Date"]), 2, 1
    ), 0
)

# 4. Phân loại LOB_2 theo logic nghiệp vụ (Lodging/NonLodging, Chat/Voice, Nesting/Production)
hc_extend["LOB_2"] = np.select(
    [
        (hc_extend["Detail Status"].str.contains("Production", na=False)) & (hc_extend["LOB"] == "Lodging") & (hc_extend["Role"].str.contains("Chat", na=False)),
        (hc_extend["Detail Status"].str.contains("Production", na=False)) & (hc_extend["LOB"] == "Non_Lodging") & (hc_extend["Role"].str.contains("Chat", na=False)),
        (hc_extend["Detail Status"].isin(["Lodging Extended Nesting", "Lodging Nesting"])) & (hc_extend["Role"].str.contains("Chat", na=False)),
        (hc_extend["Detail Status"].isin(["Non Lodging Extended Nesting", "Non Lodging Nesting"])) & (hc_extend["Role"].str.contains("Chat", na=False)),
        (hc_extend["Detail Status"].str.contains("Production", na=False)) & (hc_extend["LOB"] == "Lodging") & (hc_extend["Role"].str.contains("Voice", na=False)),
        (hc_extend["Detail Status"].str.contains("Production", na=False)) & (hc_extend["LOB"] == "Non_Lodging") & (hc_extend["Role"].str.contains("Voice", na=False)),
        (hc_extend["Detail Status"].isin(["Lodging Extended Nesting", "Lodging Nesting"])) & (hc_extend["Role"].str.contains("Voice", na=False)),
        (hc_extend["Detail Status"].isin(["Non Lodging Extended Nesting", "Non Lodging Nesting"])) & (hc_extend["Role"].str.contains("Voice", na=False)),
    ],
    [
        "Lodging Chat",
        "Non Lodging Chat",
        "Lodging Nesting Chat",
        "Non Lodging Nesting Chat",
        "Lodging Voice",
        "Non Lodging Voice",
        "Lodging Nesting Voice",
        "Non Lodging Nesting Voice"
    ],
    default=None
).astype(object)

hc_extend["LOB_3"] = hc_extend["LOB_2"]

# 5. Chuẩn hóa các trường đặc biệt (xử lý giá trị thiếu, trùng logic dashboard)
hc_extend[["Detail Status", "AON", "LG Tenure", "NL Tenure", "Concurrency", "Status"]] = hc_extend[["Detail Status", "AON", "LG Tenure", "NL Tenure", "Concurrency", "Status"]].replace({pd.NA: np.nan})

# 6. Đánh dấu dòng "Excluded" cho các trường hợp không cần tính toán performance (ví dụ còn đang training, unavailable)
hc_extend["Performance_Calculation"] = np.where(
    hc_extend["Detail Status"].isna(), "Excluded",
    np.where(hc_extend["Detail Status"].isin(["CCT Training", "Lodging Training", "Non Lodging Training", "Unavailable"]), "Excluded", "Included")
)

# 7. Loại bỏ các dòng chưa vào bất kỳ pipeline nào
hc_extend = hc_extend[hc_extend["Detail Status"] != "Unavailable"]

# 8. Loại dòng trùng lặp Date, IEX ID, Employee Name (snapshot unique)
hc_extend = hc_extend.drop_duplicates(subset=["Date", "IEX ID", "Employee Name"])

# 9. Enrich với bảng date_format để bổ sung thông tin tuần/tháng
hc_extend = pd.merge(hc_extend, date_format, on="Date", how="left")

# 10. Đặt lại Stage (In Production/Nesting/Training)
hc_extend["Stage"] = np.select(
    [
        hc_extend["Detail Status"].str.contains("Production", na=False),
        hc_extend["Detail Status"].str.contains("Training", na=False),
        hc_extend["Detail Status"].str.contains("Nesting", na=False)
    ],
    ["In Production", "In Training", "In Nesting"],
    default=None
)

# 11. Tạo cột lý do nghỉ việc cấp 3 (ATT_Reason_LV3) bằng cách lấy các từ sau thứ 4 của lý do attrition
hc_extend["ATT_Reason_LV3"] = hc_extend["Reason for Attrition"].str.split(" ").str[4:].str.join(" ")

# 12. Tính Headcount Open/Closed từng tuần/tháng, attrition và movement logic
hc_extend["HC Open By Week"] = np.where(
    (hc_extend["Date"] <= hc_extend["LWD/Movement"]) | (hc_extend["LWD/Movement"].isna()),
    np.where(hc_extend["Date"] == hc_extend["Date Start Week"], 1, 0),
    0
)

hc_extend["HC Closed By Week"] = np.where(
    (hc_extend["Date"] <= hc_extend["LWD/Movement"]) | (hc_extend["LWD/Movement"].isna()),
    np.where(hc_extend["Date"] == hc_extend["Date End Week"], 1, 0),
    0
)

hc_extend["HC Open By Month"] = np.where(
    (hc_extend["Date"] <= hc_extend["LWD/Movement"]) | (hc_extend["LWD/Movement"].isna()),
    np.where(hc_extend["Date"] == hc_extend["Date Start Month"], 1, 0),
    0
)

hc_extend["HC Closed By Month"] = np.where(
    (hc_extend["Date"] <= hc_extend["LWD/Movement"]) | (hc_extend["LWD/Movement"].isna()),
    np.where(hc_extend["Date"] == hc_extend["Date End Month"], 1, 0),
    0
)

# 13. Tính ATT/Movement, ATT, Movement: xác định ngày nhân sự nghỉ, lý do nghỉ, chuyển queue
hc_extend["ATT/Movement"] = np.where(hc_extend["Date"] == (hc_extend["LWD/Movement"] + pd.Timedelta(days=1)), 1, 0)

hc_extend["ATT"] = np.where(
    (hc_extend["ATT/Movement"] == 1) & (hc_extend["Attrition Type"].isin(["Voluntary", "Involuntary"])),
    1, 0
)

hc_extend["Movement"] = hc_extend["ATT/Movement"] - hc_extend["ATT"]

# 14. Xử lý giá trị thiếu/null và chuẩn hóa lại các field đặc biệt
hc_extend[["LOB_2", "LOB_3", "Stage", "ATT/Movement", "ATT", "Movement"]] = hc_extend[["LOB_2", "LOB_3", "Stage", "ATT/Movement", "ATT", "Movement"]].replace({pd.NA: np.nan, np.nan: 0})

# 15. Xác định trạng thái CSG (trước/sau ngày joining CSG)
hc_extend["CSG"] = np.where(
    (hc_extend["Date"] < hc_extend["CSG Joining Date"]) | (hc_extend["CSG Joining Date"].isna()),
    0, 1
)

hc_extend['HC Open By Week'] = np.where((hc_extend['Date'] <= hc_extend['LWD/Movement']) | pd.isna(hc_extend['LWD/Movement']), 
                                        np.where(hc_extend['Date'] == hc_extend['Date Start Week'], 1, 0), 0)

hc_extend['HC Closed By Week'] = np.where((hc_extend['Date'] <= hc_extend['LWD/Movement']) | pd.isna(hc_extend['LWD/Movement']), 
                                          np.where(hc_extend['Date'] == hc_extend['Date End Week'], 1, 0), 0)

hc_extend['HC Open By Month'] = np.where((hc_extend['Date'] <= hc_extend['LWD/Movement']) | pd.isna(hc_extend['LWD/Movement']), 
                                         np.where(hc_extend['Date'] == hc_extend['Date Start Month'], 1, 0), 0)

hc_extend['HC Closed By Month'] = np.where((hc_extend['Date'] <= hc_extend['LWD/Movement']) | pd.isna(hc_extend['LWD/Movement']), 
                                           np.where(hc_extend['Date'] == hc_extend['Date End Month'], 1, 0), 0)

hc_extend['ATT/Movement'] = np.where(hc_extend['Date'] == (hc_extend['LWD/Movement'] + pd.Timedelta(days=1)), 1, 0)

hc_extend['ATT'] = np.where((hc_extend['ATT/Movement'] == 1) & (hc_extend['Attrition Type'].isin(['Voluntary', 'Involuntary'])), 1, 0)

hc_extend['Movement'] = hc_extend['ATT/Movement'] - hc_extend['ATT']

hc_extend['LOB_2'] = hc_extend['LOB_2'].replace({pd.NA: None})
hc_extend['LOB_3'] = hc_extend['LOB_3'].replace({pd.NA: None})
hc_extend['Stage'] = hc_extend['Stage'].replace({pd.NA: None})
hc_extend['ATT/Movement'] = hc_extend['ATT/Movement'].fillna(0)
hc_extend['ATT'] = hc_extend['ATT'].fillna(0)
hc_extend['Movement'] = hc_extend['Movement'].fillna(0)

hc_extend['CSG'] = np.where((hc_extend['Date'] < hc_extend['CSG Joining Date']) | pd.isna(hc_extend['CSG Joining Date']), 0, 1)

# 16. Đặt lại thứ tự các cột theo đúng chuẩn bảng báo cáo/dashboard
column_order = [
    "Year", "Month", "Site", "Queue Group", "Week Begin", "Date Start Month", "Date End Month", 
    "Date Start Week", "Date End Week", "Week", "Day", "Date", "OracleID", "People ID", "IEX ID", "Employee Name", 
    "Alias", "Gender", "Designation","Grade", "LOB", "Role", "Multiple Chat Effective Date", "Primary role", "Secondary role", 
    "TL ID", "Supervisor Name", "Manager/OM Name", "Email Id", "Wave", "CCT Training", "Lodging Training", 
    "Lodging Nesting", "Lodging Extended Nesting", "CSG Joining Date", "Lodging Production", "NonLodging Training", 
    "NonLodging Nesting", "NonLodging Extended Nesting", "NonLodging Production", "Production", "LWD/Movement", 
    "Reason for Attrition", "Attrition Type", "ATT_Reason_LV3", "Detail Status", "AON", "LG Tenure", "NL Tenure", 
    "Concurrency", "Status", "LOB_2", "LOB_3", "Performance_Calculation", "Stage", "HC Open By Week", 
    "HC Closed By Week", "HC Open By Month", "HC Closed By Month", "ATT/Movement", "ATT", "Movement", "CSG"
]

hc_extend = hc_extend[column_order]
hc_extend.reset_index(drop=True, inplace=True)

In [9]:
hc_extend['Date']

0        2023-12-01
1        2023-12-01
2        2023-12-01
3        2023-12-01
4        2023-12-01
            ...    
270318   2025-06-29
270319   2025-06-29
270320   2025-06-29
270321   2025-06-29
270322   2025-06-29
Name: Date, Length: 270323, dtype: datetime64[ns]

In [10]:
mini_team = pd.read_excel(folder_paths['hc_staffing'], sheet_name="Mini Team")

columns_to_select = [
    "Mini TL",
    "Mini TL - Email",
    "Emp Email",
    "Role",
    "Active",
    "Mini TL - Short Name",
    "Effective Date"
]

mini_team = mini_team[columns_to_select]
mini_team = mini_team.rename(columns={"Emp Email": "Email Id"})

mini_team["Effective Date"] = pd.to_datetime(mini_team["Effective Date"], errors='coerce')
hc_extend["Date"] = pd.to_datetime(hc_extend["Date"], errors='coerce')

hc_extend_email_date = hc_extend[["Date", "Email Id"]].drop_duplicates()

merge_key = pd.merge(
    hc_extend_email_date,
    mini_team[["Email Id", "Effective Date"]],
    on="Email Id",
    how="left"
)

merge_key = merge_key[merge_key["Effective Date"] <= merge_key["Date"]]

merge_key = merge_key.sort_values(["Email Id", "Date", "Effective Date"])
merge_key = merge_key.drop_duplicates(subset=["Email Id", "Date"], keep="last")

merge_mini_team = pd.merge(
    merge_key,
    mini_team[["Email Id", "Effective Date", "Mini TL - Email", "Mini TL - Short Name"]],
    on=["Email Id", "Effective Date"],
    how="left"
).sort_values(by="Date")

hc_extend = pd.merge(
    hc_extend,
    merge_mini_team[["Email Id", "Date", "Mini TL - Email", "Mini TL - Short Name", "Effective Date"]],
    on=["Email Id", "Date"],
    how="left"
)

hc_extend = hc_extend.rename(columns={"Effective Date": "Mini TL Start Date"})

hc_extend

,Year,Month,Site,Queue Group,Week Begin,Date Start Month,Date End Month,Date Start Week,Date End Week,Week,...,HC Closed By Week,HC Open By Month,HC Closed By Month,ATT/Movement,ATT,Movement,CSG,Mini TL - Email,Mini TL - Short Name,Mini TL Start Date
0,2023,Dec-23,-,-,WB2711,2023-12-01,2023-12-31,2023-11-27,2023-12-03,49,...,0,1,0,0,0,0,0,NaN,NaN,NaT
1,2023,Dec-23,QTSC,-,WB2711,2023-12-01,2023-12-31,2023-11-27,2023-12-03,49,...,0,1,0,0,0,0,0,NaN,NaN,NaT
2,2023,Dec-23,QTSC,-,WB2711,2023-12-01,2023-12-31,2023-11-27,2023-12-03,49,...,0,1,0,0,0,0,0,NaN,NaN,NaT
3,2023,Dec-23,ONEHUB,CSG,WB2711,2023-12-01,2023-12-31,2023-11-27,2023-12-03,49,...,0,1,0,0,0,0,0,NaN,NaN,NaT
4,2023,Dec-23,ONEHUB,CSG,WB2711,2023-12-01,2023-12-31,2023-11-27,2023-12-03,49,...,0,1,0,0,0,0,0,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270378,2025,Jun-25,ONEHUB,CSG,WB2306,2025-06-01,2025-06-30,2025-06-23,2025-06-29,26,...,0,0,0,0,0,0,0,NaN,NaN,NaT
270379,2025,Jun-25,ONEHUB,CSG,WB2306,2025-06-01,2025-06-30,2025-06-23,2025-06-29,26,...,0,0,0,0,0,0,0,NaN,NaN,NaT
270380,2025,Jun-25,FLEMINGTON,CSG,WB2306,2025-06-01,2025-06-30,2025-06-23,2025-06-29,26,...,1,0,0,0,0,0,1,hoangkhoi.nguyen@concentrix.com,Jimi,2025-05-01
270381,2025,Jun-25,ONEHUB,CSG,WB2306,2025-06-01,2025-06-30,2025-06-23,2025-06-29,26,...,0,0,0,0,0,0,0,tiendat.nguyen1@concentrix.com,Ian,2025-05-01


In [11]:
dtype_map = {
    "Year": 'Int64',
    "Month": 'string',
    "Site": 'string',
    "Queue Group": 'string',
    "Week Begin": 'string',
    "Week": 'Int64',
    "Day": 'string',
    "OracleID": 'Int64',
    "People ID": 'Int64',
    "IEX ID": 'Int64',
    "Employee Name": 'string',
    "Alias": 'string',
    "Gender": 'string',
    "Designation": 'string',
    "Grade":'string',
    "LOB": 'string',
    "Role": 'string',
    "Primary role": 'string',
    "Secondary role": 'string',
    "TL ID": 'Int64',
    "Supervisor Name": 'string',
    "Manager/OM Name": 'string',
    "Email Id": 'string',
    "Reason for Attrition": 'string',
    "Attrition Type": 'string',
    "ATT_Reason_LV3": 'string',
    "Detail Status": 'string',
    "LG Tenure": 'string',
    "NL Tenure": 'string',
    "Status": 'string',
    "LOB_2": 'string',
    "LOB_3": 'string',
    "Performance_Calculation": 'string',
    "Stage": 'string',
    "Wave": 'object',
    "AON": 'Int64',
    "Concurrency": 'Int64',
    "HC Open By Week": 'Int64',
    "HC Closed By Week": 'Int64',
    "HC Open By Month": 'Int64',
    "HC Closed By Month": 'Int64',
    "ATT/Movement": 'Int64',
    "ATT": 'Int64',
    "Movement": 'Int64',
    "CSG": 'Int64',
    "Mini TL - Email":"string", 
    "Mini TL - Short Name":"string"
}

date_cols = [
    "Date Start Month", "Date End Month", "Date Start Week", "Date End Week",
    "Date", "Multiple Chat Effective Date","CCT Training", "Lodging Training", "Lodging Nesting",
    "Lodging Extended Nesting", "CSG Joining Date", "Lodging Production",
    "NonLodging Training", "NonLodging Nesting", "NonLodging Extended Nesting",
    "NonLodging Production", "Production", "LWD/Movement","Mini TL Start Date"
]

for col in date_cols:
    if col in hc_extend.columns:
        hc_extend[col] = pd.to_datetime(hc_extend[col], errors='coerce').dt.date

for col in hc_extend.columns:
    if col in dtype_map:
        if dtype_map[col] == 'Int64':
            hc_extend[col] = pd.to_numeric(hc_extend[col], errors='coerce').astype('Int64')
        elif dtype_map[col] == 'string':
            hc_extend[col] = hc_extend[col].astype('string')
        else:
            hc_extend[col] = hc_extend[col].astype(dtype_map[col])

print(hc_extend.dtypes)

hc_extend


Year                             Int64
Month                   string[python]
Site                    string[python]
Queue Group             string[python]
Week Begin              string[python]
                             ...      
Movement                         Int64
CSG                              Int64
Mini TL - Email         string[python]
Mini TL - Short Name    string[python]
Mini TL Start Date              object
Length: 66, dtype: object


,Year,Month,Site,Queue Group,Week Begin,Date Start Month,Date End Month,Date Start Week,Date End Week,Week,...,HC Closed By Week,HC Open By Month,HC Closed By Month,ATT/Movement,ATT,Movement,CSG,Mini TL - Email,Mini TL - Short Name,Mini TL Start Date
0,2023,Dec-23,-,-,WB2711,2023-12-01,2023-12-31,2023-11-27,2023-12-03,49,...,0,1,0,0,0,0,0,<NA>,<NA>,NaT
1,2023,Dec-23,QTSC,-,WB2711,2023-12-01,2023-12-31,2023-11-27,2023-12-03,49,...,0,1,0,0,0,0,0,<NA>,<NA>,NaT
2,2023,Dec-23,QTSC,-,WB2711,2023-12-01,2023-12-31,2023-11-27,2023-12-03,49,...,0,1,0,0,0,0,0,<NA>,<NA>,NaT
3,2023,Dec-23,ONEHUB,CSG,WB2711,2023-12-01,2023-12-31,2023-11-27,2023-12-03,49,...,0,1,0,0,0,0,0,<NA>,<NA>,NaT
4,2023,Dec-23,ONEHUB,CSG,WB2711,2023-12-01,2023-12-31,2023-11-27,2023-12-03,49,...,0,1,0,0,0,0,0,<NA>,<NA>,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270378,2025,Jun-25,ONEHUB,CSG,WB2306,2025-06-01,2025-06-30,2025-06-23,2025-06-29,26,...,0,0,0,0,0,0,0,<NA>,<NA>,NaT
270379,2025,Jun-25,ONEHUB,CSG,WB2306,2025-06-01,2025-06-30,2025-06-23,2025-06-29,26,...,0,0,0,0,0,0,0,<NA>,<NA>,NaT
270380,2025,Jun-25,FLEMINGTON,CSG,WB2306,2025-06-01,2025-06-30,2025-06-23,2025-06-29,26,...,1,0,0,0,0,0,1,hoangkhoi.nguyen@concentrix.com,Jimi,2025-05-01
270381,2025,Jun-25,ONEHUB,CSG,WB2306,2025-06-01,2025-06-30,2025-06-23,2025-06-29,26,...,0,0,0,0,0,0,0,tiendat.nguyen1@concentrix.com,Ian,2025-05-01


In [12]:
for Week_Monday, group in hc_extend.groupby('Month'):
    file_name = f'{Week_Monday}.xlsx'
    file_path = os.path.join(folder_paths["hc_extend_by_week"], file_name)
    group.to_excel(file_path, index=False)
